## SIT796 Reinforcement Learning

Distinction Task 2.2 - Exact Policy Iteration Implementation for MDPs  
**Angus Maiden** | ID 220595465

This task implements the policy iteration method of finding an optimal policy that solves a Markov Decision Process (MDP), as outlined in Task 3.1C

In [1]:
# Import the necessary libraries.
import gym
import numpy as np
import matplotlib.pyplot as plt

# Invoke the model maze environment  from Task 1.2C.
env = gym.make('rl_gym_maze:rl-gym-maze-v0')

### Initialising variables

In [2]:
# Initialise environment.
env.reset()

# Initialise dictionary to store flattened index of (x,y) coordinates.
states = {}
count = 0
for i in range(10):
    for j in range(10):
        states[i, j] = count
        count+=1

# Initialise V table, max_iterations and gamma.
V = np.zeros(100)
max_iterations = 1000
gamma = 1

# Initialise other variables.
next_state = np.zeros([4,2])
next_reward = np.zeros(4)
next_done = np.zeros(4, dtype=bool)
next_V = np.zeros(4)

### Policy Evaluation

In [3]:
def policy_evaluation():
    
    # Initalise episode counter.
    episodes = 0
    
    # Policy evluation limited to 1000 sweeps (limited to save computational time).
    for i in range(max_iterations):
    
        # Initialise delta.
        delta = 0
        
        # Initalise state.
        env.reset()
        
        # Iterate though each state.
        for x in range(10):
            for y in range(10):
                 
                # Get flattened current state.
                current_state = states[x, y]
                
                # Initalise v.
                v = 0
            
                # Loop over each action.
                for a in range(4):
                
                    # Populate a list of successor states and rewards for each eaction.
                    next_state[a], next_reward[a], next_done[a], info = env.step(a)
                    next_V[a] = states[next_state[a][0], next_state[a][1]]
                
                    # Calculate V(s) using Bellman equation:
                    v += 0.25 * (next_reward[a] + gamma * V[int(next_V[a])])
                
                    # This is a lookahead, so reset state to before the step.
                    env.location = (x,y)
            
                # Get delta value, which is the change in V(s).
                delta = max(delta, np.abs(V[current_state] - v))
            
                # Update V(s) with calculated value.
                V[current_state] = v
            
                # The state value of the terminal state is always 0.
                V[99] = 0
                
                # Update environment state to next in sweep.
                env.location = (x,y+1)
                if env.location[1] == 10:
                    env.location = (x+1, 0)
                
        # Increment episode counter and display progress indicator.
        episodes += 1
        print(f'\rPolicy Evaluation: Episode {episodes}', end='')
        
        # If the change is state value is sufficiently small, exit the loop.
        if delta < 0.1:
            # Print the results and return the new V table.
            print(f'\rPolicy evaluated in {episodes} episodes.')
            return V
    
    # Print the results and return the new V table.
    print(f'\rPolicy evaluated in {episodes} episodes.')
    return V

### One step look-ahead function

In [4]:
def action_values(state):
    
    # Check that 'state' argument is a tuple, for environment compatability.
    assert isinstance(state, tuple), '"state" argument should be a tuple'
    
    # Initialise Q.
    Q = np.zeros(4)
    
    # Set environment state to 'state' argument.
    env.location = state
    
    # Loop over each action.
    for a in range(4):
           
        # Populate a list of successor states and rewards for each eaction.
        next_state[a], next_reward[a], next_done[a], info = env.step(a)
        next_V[a] = states[next_state[a][0], next_state[a][1]]
                
        # Update Q(s,a) for each a using Bellman equation:
        Q[a] = (next_reward[a] + gamma * V[int(next_V[a])])
                
        # This is a lookahead, so reset state to before the step.
        env.location = state
    
    # Return a 4-tuple of the action-values at 'state'.
    return Q

### Policy improvement

In [5]:
# Define initial policy, P, as random (p(s,a) = 0.25 for all s).
P = np.ones([100, 4]) / 4

# Initialise iterations counter.
iterations = 0

# Policy iteration limited to 100 iterations (hoping to optimise before limit).
for i in range(100):
    
    # Initialise 'stable' value for determining if policy is optmised.
    stable = True
    
    # Get V table from policy evaluation.
    V = policy_evaluation()
    
    # Iterate though each state.
    for x in range(10):
        for y in range(10):
            
            # Get flattened current state.
            current_state = states[x, y]
            
            # Select the best action (argmax) under current policy, breaking ties randomly.
            old_action = int(np.random.choice(np.flatnonzero(P[current_state] == P[current_state].max())))
            
            # Get next action values at current state.
            Q = action_values((x,y))
            
            # Select best action (argmax) based on new action values, breaking ties randomly.
            new_action = int(np.random.choice(np.flatnonzero(Q == Q.max())))
            
            # Check if action under the new policy is different than the old.
            if old_action != new_action:
                stable = False
                P[current_state] = np.eye(4)[new_action]
            
            # Update environment state to next in sweep.
            env.location = (x,y+1)
            if env.location[1] == 10:
                env.location = (x+1, 0)
    
    # Increment iteration counter and display progress indicator.
    iterations += 1
    print(f'\nPolicy Improvement: Iteration {iterations}\n')
    
    # If no change in policy, we have an optimal policy (exit the loop).
    if stable:
        break

# Print the results, including optimal state values as an array.
print(f'Found optimal policy in {iterations} iterations.\n')
print(f'Grid view of the optimal policy:\n')
np.set_printoptions(precision = 2, suppress = True, linewidth = 100)
print(V.reshape([10,10]))

Policy evaluated in 1000 episodes.

Policy Improvement: Iteration 1

Policy evaluated in 1000 episodes.

Policy Improvement: Iteration 2

Policy evaluated in 1000 episodes.

Policy Improvement: Iteration 3

Policy evaluated in 1000 episodes.

Policy Improvement: Iteration 4

Policy evaluated in 1000 episodes.

Policy Improvement: Iteration 5

Policy evaluated in 1000 episodes.

Policy Improvement: Iteration 6

Found optimal policy in 6 iterations.

Grid view of the optimal policy:

[[-1380.39 -1376.4  -1368.41 -1356.43 -1340.46 -1323.66 -1294.89 -1254.12 -1230.25 -1198.88]
 [-1505.85 -1512.17 -1514.49 -1512.82 -1337.3  -1331.65 -1302.87 -1233.25 -1233.75 -1163.52]
 [-1495.54 -1467.89 -1489.52 -1507.16 -1330.13 -1335.65 -1306.87 -1207.87 -1128.66 -1124.17]
 [-1481.25 -1442.26 -1519.14 -1515.15 -1318.98 -1322.97 -1194.48 -1178.5  -1129.16 -1076.33]
 [-1462.95 -1412.65 -1379.04 -1341.44 -1299.84 -1087.81 -1206.46 -1214.45 -1218.44  -971.65]
 [-1440.67 -1452.65 -1460.63 -1464.63 -1235.11 -

### Running an episode with optimal policy

In [6]:
# Initalise environment and other variables.
state = env.reset()
done = False
counter = 0
total_reward = 0

# Limit episode to 1000 steps (hoping to finish the task before limit).
for step in range(1000):
    
     # Get flattened current state.
    current_state = states[state[0],state[1]]
    
    # Select next action based on optimal policy.
    action = int(np.random.choice(np.flatnonzero(P[current_state] == P[current_state].max())))
    
    # Get state, reward and done variables from environment 'step' function.
    state, reward, done, info = env.step(action)
    
    # Update counter and reward tracking variables.
    counter += 1
    total_reward += reward
    
    # Display current location as agent moves through maze.
    print(f'Step: {counter} | Location: {state} | Reward received: {reward}')
    
    # If we reach the goal, end the episode and display results.
    if done:
        print(f'\nReached the goal in {counter} steps with optimal policy.')
        print(f'Total episode reward: {total_reward}')
        break
    
    if step == 999:
        print(f'Failed to reach goal within limit. Not an optimal policy.')

Step: 1 | Location: [0 1] | Reward received: -1
Step: 2 | Location: [0 2] | Reward received: -1
Step: 3 | Location: [0 3] | Reward received: -1
Step: 4 | Location: [0 4] | Reward received: -1
Step: 5 | Location: [0 5] | Reward received: -1
Step: 6 | Location: [0 6] | Reward received: -1
Step: 7 | Location: [0 7] | Reward received: -1
Step: 8 | Location: [0 8] | Reward received: -1
Step: 9 | Location: [0 9] | Reward received: -1
Step: 10 | Location: [1 9] | Reward received: -1
Step: 11 | Location: [2 9] | Reward received: -1
Step: 12 | Location: [3 9] | Reward received: -1
Step: 13 | Location: [4 9] | Reward received: -1
Step: 14 | Location: [5 9] | Reward received: -1
Step: 15 | Location: [6 9] | Reward received: -1
Step: 16 | Location: [7 9] | Reward received: -1
Step: 17 | Location: [8 9] | Reward received: -1
Step: 18 | Location: [9 9] | Reward received: 0

Reached the goal in 18 steps with optimal policy.
Total episode reward: -17
